# Sentiment classification using Sentiment Neuron

In [5]:
import sys
sys.path.append('/home/FRACTAL/swati.tiwari/generating-reviews-discovering-sentiment-master')

from encoder import Model
import pandas as pd
import numpy as np
import csv
import time
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
import pickle
import warnings
import os
import random
import utils
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
plt.style.use('ggplot')
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2
%matplotlib inline
pd.set_option('display.max_colwidth' , -1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
aspect_lvl_data = pd.read_csv('Aspect-Based-Sentiment-Analysis/output/aspect_level_test_data.csv')

Fork follwoing library to utilize sentiment neuron model - https://github.com/rakeshchada/generating-reviews-discovering-sentiment/blob/master/Sentiment-Neuron-Yelp.ipynb

In [3]:
model = Model()
test_features = model.transform(aspect_lvl_data['sp_text'])
test_features = test_features[0]
sentiment_neuron_index = 2388
aspect_lvl_data['prediction'] = test_features[: , sentiment_neuron_index]

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
dim is deprecated, use axis instead
311.371 seconds to transform 725 examples


Sentiment neuron give result from -2 till 2 , If the result is between 0 - 0.25 then we call it Neutral sentiment , if it is smaller than 0 , then we call it as Negative sentiment other wise Positive Sentiment

In [9]:
def map_pred(i):
    if i >=0.25: return 'positive' ## positive category
    elif i < 0: return 'negative'  ## negative category
    else:    return 'neutral' ## neutral category
    
aspect_lvl_data['y_pred'] = aspect_lvl_data['prediction'].apply(lambda x: map_pred(x))

In [10]:
aspect_lvl_data['fn_polarity'] = aspect_lvl_data['y_pred']
aspect_lvl_data['fn_aspect'].fillna('anecdotes/miscellaneous' , inplace = True)

In [13]:
"""
loading the test data with ground truth value of aspects and setiments.
"""

import ast
test_grp_df  = pd.read_csv('Aspect-Based-Sentiment-Analysis//data/resturant_test_stratified_grouped.csv')
test_grp_df['aspects'] = test_grp_df['aspects'].apply(lambda x: ast.literal_eval(x))
test_grp_df['polarities'] = test_grp_df['polarities'].apply(lambda x: ast.literal_eval(x))
test_grp_df.reset_index(inplace=True)
test_grp_df.rename(columns={'index': 'id'} , inplace=True)

### Following steps are perfromed to merge the ground truth data frame with aspect and sentiment prediction 

In [18]:
df_aspect_pred = aspect_lvl_data.groupby(['id'])['fn_aspect'].apply(lambda x : ' '.join(x)).reset_index()
df_polarity_pred = aspect_lvl_data.groupby(['id'])['fn_polarity'].apply(lambda x : ' '.join(x)).reset_index()
df_pred = pd.merge(df_aspect_pred , df_polarity_pred , on = ['id'])
test_grp_df.rename(columns={'index': 'id'} , inplace=True)
test_grp_pred_df = pd.merge(test_grp_df ,df_pred , on = ['id'])
test_grp_pred_df['fn_aspect']  = test_grp_pred_df['fn_aspect'].apply(lambda x  : x.split(' '))
test_grp_pred_df['fn_polarity']  = test_grp_pred_df['fn_polarity'].apply(lambda x  : x.split(' '))

In [21]:
asp_df = test_grp_pred_df.aspects.apply(pd.Series).merge(test_grp_pred_df , right_index = True , left_index = True)\
.drop(['aspects' , 'polarities' , 'text' , 'length' , 'fn_aspect' , 'fn_polarity'] ,axis = 1)\
.melt(id_vars = ['id', 'ind']).drop(['variable'] , axis = 1).dropna()

polarity_df = test_grp_pred_df.polarities.apply(pd.Series).merge(test_grp_pred_df , right_index = True , left_index = True)\
.drop(['aspects' , 'polarities' , 'text' , 'length' , 'fn_aspect' , 'fn_polarity'] ,axis = 1)\
.melt(id_vars = ['id', 'ind']).drop(['variable'] , axis = 1).dropna()

test_ground_truth = pd.merge(asp_df , polarity_df[['value']] , left_index = True , right_index = True ,suffixes=('_aspects' , '_polarities'))
test_ground_truth.rename(columns={'value_aspects' : 'aspects' , 'value_polarities':'polarities'} , inplace=True)

In [22]:
asp_df = test_grp_pred_df.fn_aspect.apply(pd.Series).merge(test_grp_pred_df , right_index = True , left_index = True)\
.drop(['aspects' , 'polarities' , 'text' , 'length' , 'fn_aspect' , 'fn_polarity' , 'ind'] ,axis = 1)\
.melt(id_vars = ['id']).drop(['variable'] , axis = 1).dropna()
polarity_df = test_grp_pred_df.fn_polarity.apply(pd.Series).merge(test_grp_pred_df , right_index = True , left_index = True)\
.drop(['aspects' , 'polarities' , 'text' , 'length' , 'fn_aspect' , 'fn_polarity' , 'ind'] ,axis = 1)\
.melt(id_vars = ['id']).drop(['variable'] , axis = 1).dropna()

test_pred_df = pd.merge(asp_df , polarity_df[['value']] , left_index = True , right_index = True ,suffixes=('_aspects' , '_polarities'))
test_pred_df.rename(columns={'value_aspects' : 'aspects' , 'value_polarities':'polarities'} , inplace=True)

In [23]:
test_ground_truth.sort_values(by = ['id'] , inplace=True)
test_pred_df.sort_values(by = ['id'] , inplace=True)
test_ground_truth.drop_duplicates(['id', 'aspects'] , inplace=True)
test_pred_df.drop_duplicates(['id', 'aspects'] , inplace=True)
test_ground_truth.shape , test_pred_df.shape

((867, 4), (717, 3))

#### We are observing results at text X aspect level for test data.  The following result combine aspect prediction , sentence segmentation and sentiment prediction

In [24]:
aspect_sentiment_lvl = pd.merge(test_ground_truth , test_pred_df , on= ['id'  , 'aspects']  , how = 'left' ,suffixes=('' , '_pred'))

In [37]:
print('Aspect prediction accuracy' , 1- (test_ground_truth.shape[0]- test_pred_df.shape[0])/test_ground_truth.shape[0])

Aspect prediction accuracy 0.8269896193771626


In [32]:
print( 'Overall sentiment accuracy, ' , np.mean(aspect_sentiment_lvl['polarities']== aspect_sentiment_lvl['polarities_pred']))
subset = aspect_sentiment_lvl[aspect_sentiment_lvl['polarities_pred'].notnull()]
print('Accuracy for only correctly predicted aspect categories ', np.mean(subset['polarities']== subset['polarities_pred']))

Overall sentiment accuracy,  0.45674740484429066
Accuracy for only correctly predicted aspect categories  0.6723259762308998


In [35]:
from sklearn.metrics import f1_score ,  recall_score , precision_score , confusion_matrix
print( 'F1 score' , f1_score(subset['polarities'] , subset['polarities_pred'] , average='macro'))
print(confusion_matrix(subset['polarities'] , subset['polarities_pred'] ))

F1 score 0.5839993775287892
[[ 94  25  18]
 [ 32  34  21]
 [ 43  54 268]]
